# CNN Training

In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Move to your current working directory
%cd drive/MyDrive/image_processing

Mounted at /content/drive
/content/drive/MyDrive/image_processing


In [ ]:
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device', device)

Device cuda


In [ ]:
import os
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision.transforms as T
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader

## Load Images

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, image_dir, transform):
    super().__init__()
    self.image_dir = image_dir
    self.transform = transform
    self.images = os.listdir(self.image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, i):
    image_path = os.path.join(self.image_dir, self.images[i])
    image = np.array(Image.open(image_path).convert('L'))
    image = self.transform(image)
    label = int(self.images[i][1])
    return (image, label)

In [ ]:
training_data = CustomDataset('img_span12/training', T.ToTensor())
testing_data = CustomDataset('img_span12/testing', T.ToTensor())

In [ ]:
print('training images:', len(training_data))
print('training images:', len(testing_data))

training images: 7746
training images: 2350


In [ ]:
print(training_data[0])
print(testing_data[0])

(tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 0., 1., 0.]]]), 0)
(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 1., 0.]]]), 0)


## Build CNN Model

In [ ]:
class CNNModel(nn.Module):
  # Input: [N, 1, 96, 180]
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=(5, 3), stride=(3, 1), dilation=(3, 1), padding=(6, 1)), # N x 64 x 32 x 180
        nn.BatchNorm2d(64, affine=True),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2, 1)), # N x 64 x 16 x 180
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=(5, 3), stride=(1, 1), dilation=(1, 1), padding=(2, 1)), # N x 128 x 16 x 180
        nn.BatchNorm2d(128, affine=True),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2, 1)), # N x 128 x 8 x 180
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size=(5, 3), stride=(1, 1), dilation=(1, 1), padding=(2, 1)), # N x 256 x 8 x 180
        nn.BatchNorm2d(256, affine=True),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2, 1)), # N x 256 x 4 x 180
    )
    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(0.5)
    self.fc1 = nn.Linear(256*4*180, 10)
    self.fc2 = nn.Linear(256*4*180, 2)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.flatten(x)
    x = self.dropout(x)
    out1 = self.fc1(x)
    out2 = self.fc2(x)
    return out1, out2

In [ ]:
model = CNNModel()
print(model)

model = model.cuda()

CNNModel(
  (layer1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 3), stride=(3, 1), padding=(6, 1), dilation=(3, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(5, 3), stride=(1, 1), padding=(2, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(5, 3), stride=(1, 1), padding=(2, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Drop

## Training

In [ ]:
# Constants for model training process
BATCH_SIZE = 128
NUM_EPOCHS = 10
PRINT_EVERY = 10

In [ ]:
train_loader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
next(iter(train_loader))

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 1., 0.],
           [0., 0., 0.,  ..., 0., 1., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 1., 0.],
           [0., 1., 0.,  ..., 0., 1., 0.],
           [0., 1., 0.,  ..., 0., 1., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 1., 0.,  ..., 0., 0., 0.],
           [0., 1., 0.,  ..., 0., 1., 0.],
           [0., 1., 0.,  ..., 0., 1., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [1., 1., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 1., 0.,  ..., 0., 1., 0.],
           [0., 1., 0.,  ..., 0., 1., 0.],
           [0., 1., 0.,  ..., 0., 1., 0.]]],
 
 
         ...,
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ..

In [ ]:
# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
for epoch in range(NUM_EPOCHS):
  training_acc = 0
  total = 0
  for counter, (x, y) in enumerate(train_loader):
    model.train()
    x, y = x.to(device), y.to(device)
    _, scores = model(x)
    loss = loss_function(scores, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    training_acc += scores.max(1)[1].eq(y).sum().item()
    total += y.size(0)
    if counter % PRINT_EVERY == 0:
      print(f'Epoch[{epoch+1}], Batch[{counter}], Loss: {loss.item()}, Training acc: {training_acc/total}')
  print(f'Epoch[{epoch+1}] completed. Training acc: {training_acc/total}')

Epoch[1], Batch[0], Loss: 0.7167148590087891, Training acc: 0.4453125
Epoch[1], Batch[10], Loss: 0.8445115685462952, Training acc: 0.5397727272727273
Epoch[1], Batch[20], Loss: 0.8601365685462952, Training acc: 0.5260416666666666
Epoch[1], Batch[30], Loss: 0.7116991281509399, Training acc: 0.5274697580645161
Epoch[1], Batch[40], Loss: 0.7741991281509399, Training acc: 0.5268673780487805
Epoch[1], Batch[50], Loss: 0.8913865685462952, Training acc: 0.5280330882352942
Epoch[1], Batch[60], Loss: 0.7526555061340332, Training acc: 0.5307255357603925
Epoch[1] completed. Training acc: 0.5307255357603925
Epoch[2], Batch[0], Loss: 0.7741990685462952, Training acc: 0.5390625
Epoch[2], Batch[10], Loss: 0.6960741877555847, Training acc: 0.5383522727272727
Epoch[2], Batch[20], Loss: 0.7898240685462952, Training acc: 0.5398065476190477
Epoch[2], Batch[30], Loss: 0.8210740685462952, Training acc: 0.5307459677419355
Epoch[2], Batch[40], Loss: 0.7429491281509399, Training acc: 0.5346798780487805
Epoch[2

## Output Result

In [ ]:
class OutputDataset(Dataset):
  def __init__(self, image_dir, transform):
    super().__init__()
    self.image_dir = image_dir
    self.transform = transform
    self.images = os.listdir(self.image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, i):
    image_path = os.path.join(self.image_dir, self.images[i])
    image = np.array(Image.open(image_path).convert('L'))
    image = self.transform(image)
    label = int(self.images[i][1])
    stock = self.images[i].split('_')[1]
    date = self.images[i].split('_')[2].replace('.png', '')
    return (image, label, stock, date)

In [ ]:
training_data = OutputDataset('img_span12/training', T.ToTensor())
testing_data = OutputDataset('img_span12/testing', T.ToTensor())

In [ ]:
train_loader = DataLoader(training_data, batch_size=32, shuffle=False)
test_loader = DataLoader(testing_data, batch_size=32, shuffle=False)

In [ ]:
def extract_vectors(model, dataloader):
  model.eval()
  all_records = []
  total_correct = 0
  total_samples = 0

  with torch.no_grad():
    for x, y, stocks, dates in dataloader:
      x, y = x.to(device), y.to(device)
      outputs, scores = model(x)
      outputs = outputs.cpu().numpy()

      predictions = scores.max(1)[1]
      total_correct += predictions.eq(y).sum().item()
      total_samples += y.size(0)

      for i in range(len(dates)):
        record = {
            'stock': stocks[i],
            'date': dates[i],
            'vector': ','.join(map(str, outputs[i])),
            'label': y[i].item()
        }
        all_records.append(record)
    print('Acc:', total_correct/total_samples)
  return all_records

In [ ]:
# training datasets
records = extract_vectors(model, train_loader)

df = pd.DataFrame(records)
df.sort_values(by=['stock', 'date'])
print(df)

# Save the DataFrame to a CSV file
df.to_csv('output/output_vectors_training.csv', index=True)

Acc: 0.5325329202168861
     stock        date                                             vector  \
0     2912  2017-02-10  0.56787235,0.0021233691,0.1194147,-0.38013074,...   
1     2912  2017-03-02  0.5676637,-0.1419385,0.34287295,-0.086412504,-...   
2     2912  2017-03-20  0.599719,-0.1001104,0.19023514,0.36024198,-0.5...   
3     2912  2017-04-07  0.770942,0.02940203,-0.21924147,0.02415434,-0....   
4     2912  2017-04-25  0.48004425,0.091612555,-0.49475306,0.30589804,...   
...    ...         ...                                                ...   
7741  1303  2018-05-24  0.66959256,-0.21484628,-0.5411396,-0.30469495,...   
7742  1303  2018-06-11  0.79587215,-0.061968684,-0.5489692,0.3220992,-...   
7743  1303  2018-06-28  0.9078283,-0.33129203,-0.15298954,0.22587852,0...   
7744  1303  2018-07-16  0.54299307,-0.17875424,-0.7995753,-0.055917315...   
7745  1303  2018-08-01  0.73035324,-0.38349077,-0.11114937,0.15567489,...   

      label  
0         0  
1         1  
2        

In [ ]:
# testing datasets
records = extract_vectors(model, test_loader)

df = pd.DataFrame(records)
df.sort_values(by=['stock', 'date'])
print(df)

# Save the DataFrame to a CSV file
df.to_csv('output/output_vectors_testing.csv', index=True)

Acc: 0.5114893617021277
     stock        date                                             vector  \
0     2615  2022-01-10  1.0058374,0.26000896,-0.37458733,0.55736965,-0...   
1     2615  2022-01-26  0.23164836,-0.26892275,0.124287255,-0.19152172...   
2     2615  2022-02-22  0.47282687,0.10773544,-0.20609543,-0.13387553,...   
3     2615  2022-03-11  1.0380422,-0.42399195,-0.33817458,-0.38086346,...   
4     2615  2022-03-29  0.76127297,-0.84144944,0.8376868,-0.06892618,-...   
...    ...         ...                                                ...   
2345  1402  2024-04-09  0.80873096,0.017047377,0.0892035,0.29000244,-0...   
2346  1402  2024-04-25  0.34272876,0.3615686,-0.52904123,0.17904267,0....   
2347  1402  2024-05-14  0.8150911,-0.42992038,-0.24947715,-0.05869771,...   
2348  1402  2024-05-30  0.5753973,-0.38902566,-0.5974206,0.40944046,0....   
2349  1402  2024-06-18  0.72765404,-0.53896064,-0.348013,0.41687733,-0...   

      label  
0         0  
1         1  
2        